<a href="https://colab.research.google.com/github/ada-nai/nptel-PMLTF/blob/master/Week%207/TF_Assignment_7_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with an RNN

## Setup

### Import TensorFlow and other libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

tf.random.set_seed(1)

TensorFlow 2.x selected.


### Download the Shakespeare dataset

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Read the data



In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')


In [0]:
vocab = sorted(set(text))

## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Create mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

### The prediction task

### Create training examples and targets



In [0]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

### Create training batches


In [0]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding` (dim=256)
* `tf.keras.layers.LSTM` (units=1024)
* `tf.keras.layers.Dense`

In [0]:
# Write the code here to define the model
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(vocab_size = vocab_size, embedding_dim = embedding_dim, rnn_units = rnn_units, batch_size = BATCH_SIZE)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [12]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## Train the model

### Attach an optimizer, and a loss function

In [0]:
# Write code here
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [0]:
# Write code here
model.compile(optimizer='adam', loss=loss)


### Configure checkpoints

---



Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=10
fin = 40

In [18]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 10s 61ms/step - loss: 2.6613
Epoch 2/10
172/172 [==============================] - 9s 51ms/step - loss: 1.9583
Epoch 3/10
172/172 [==============================] - 9s 50ms/step - loss: 1.6911
Epoch 4/10
172/172 [==============================] - 9s 50ms/step - loss: 1.5423
Epoch 5/10
172/172 [==============================] - 9s 51ms/step - loss: 1.4548
Epoch 6/10
172/172 [==============================] - 9s 51ms/step - loss: 1.3936
Epoch 7/10
172/172 [==============================] - 9s 50ms/step - loss: 1.3489
Epoch 8/10
172/172 [==============================] - 9s 50ms/step - loss: 1.3094
Epoch 9/10
172/172 [==============================] - 9s 50ms/step - loss: 1.2747
Epoch 10/10
172/172 [==============================] - 9s 50ms/step - loss: 1.2424


In [19]:
history_1 = model.fit(dataset, epochs=fin, callbacks=[checkpoint_callback], initial_epoch= (EPOCHS))

Epoch 11/40
172/172 [==============================] - 9s 51ms/step - loss: 1.2104
Epoch 12/40
172/172 [==============================] - 9s 49ms/step - loss: 1.1789
Epoch 13/40
172/172 [==============================] - 9s 51ms/step - loss: 1.1451
Epoch 14/40
172/172 [==============================] - 9s 50ms/step - loss: 1.1112
Epoch 15/40
172/172 [==============================] - 9s 50ms/step - loss: 1.0768
Epoch 16/40
172/172 [==============================] - 9s 51ms/step - loss: 1.0406
Epoch 17/40
172/172 [==============================] - 9s 50ms/step - loss: 1.0061
Epoch 18/40
172/172 [==============================] - 9s 50ms/step - loss: 0.9701
Epoch 19/40
172/172 [==============================] - 9s 50ms/step - loss: 0.9343
Epoch 20/40
172/172 [==============================] - 9s 50ms/step - loss: 0.9000
Epoch 21/40
172/172 [==============================] - 9s 50ms/step - loss: 0.8677
Epoch 22/40
172/172 [==============================] - 9s 49ms/step - loss: 0.8379
Epoc

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [20]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_40'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


### The prediction loop


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [24]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: Know, when he's for you;
Hop stery me to speak,
And wounds all upon thy head
And dull the watch is changedoubly one, and thy father York
May them account their liberties. Let me be put to death;
I'll pace a man born to dinner, I'll swear it.

Shepherd:
Come, bring her watch'd sound, to counsel throut my holy oath:
Confess the foolish skining largues in uch laudy, one sungrone encounter men

EDWARD:
Lord Staft he did betray them.

OXFORD:
Farewell, farewell! one. Love with my nature, cousin, fear.

Third Citizen:
We are all undone! the harveham finger sounds,
Bring forth her house, the very bled
in their bed:

ISABELLA:
I am costent this debusy lord, I lose nothing
When I shall show usage of you to any rage
With flower throst letters to his part to tell,
Signior Bagna, I must eatenced:
That morth the garden rever me down to-morrow mocking now?

DORSET:
A bootless heart for be speed; what of
France; or old weeps to cheer his songs
therein throw an old trot seem your grace.

First 